# Ex2

## Q1

TBD

## Q2

TBD

## Q3

TBD

## Q4

### Get Corpus and Dictionary

In [1]:
import urllib2
import zipfile
import os

data_folder = 'corpus_combined'
corpus_file = data_folder + '.zip'

if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    
zipcontent = urllib2.urlopen('http://www2.mta.ac.il/~gideon/courses/nlp/data/' + corpus_file).read()
with open(corpus_file, 'w') as f:
    f.write(zipcontent)
zip_ref = zipfile.ZipFile(corpus_file, 'r')
zip_ref.extractall(data_folder)
zip_ref.close()
os.remove(corpus_file)

response = urllib2.urlopen('http://www2.mta.ac.il/~gideon/courses/nlp/data/word_list_20k.txt')
dictionary = response.read().split('\n')
dictionary.remove('')

### Prepare training and test set

In [2]:
import urllib2

# training
kidnapped = urllib2.urlopen('http://www.gutenberg.org/files/421/421-0.txt').read().decode('utf8')
oliver_twist = urllib2.urlopen('http://www.gutenberg.org/ebooks/730.txt.utf-8').read().decode('utf8')
training_set = [kidnapped, oliver_twist]

# validation
a_tail_of_two_cities = urllib2.urlopen('http://www.gutenberg.org/files/98/98-0.txt').read().decode('utf8')
validation_set = [a_tail_of_two_cities]

# testing
persuasion = open(data_folder + '/persuasion.txt', 'r').read()
testing_set = [persuasion]

### Hyperparameter Tuning

In [3]:
hyperparameters = {
    'lowercase': True,
    'stemming': False,
    'remove_stopwords': False
}

### Pre-Processing

In [4]:
from chapterize import Book
from nltk.tokenize import word_tokenize

OOV = 'OOV'
NUM = 'NUM'

# removing header, footer and chapters
training = ''
for book in training_set:
    training += Book(book).getContent() + ' '
validation = ''
for book in validation_set:
    validation += Book(book).getContent() + ' '
testing = ''
for book in testing_set:
    testing += Book(book).getContent() + ' '

# lowercasing
if hyperparameters['lowercase']:
    training = training.lower()
    validation = validation.lower()
    testing = testing.lower()
    dictionary = [word.lower() for word in dictionary]
    
# tokenizing
training = word_tokenize(training)
validation = word_tokenize(validation)
testing = word_tokenize(testing)

# removing stopwords
if hyperparameters['remove_stopwords']:
    from nltk.corpus import stopwords
    
    stopwords_set = set(stopwords.words('english'))
    training = [token for token in training if token not in stopwords_set]
    validation = [token for token in validation if token not in stopwords_set]
    testing = [token for token in testing if token not in stopwords_set]
    dictionary = [word for word in dictionary if word not in stopwords_set]
    
# stemming
if hyperparameters['stemming']:
    from nltk.stem import PorterStemmer
    
    ps = PorterStemmer()
    training = [ps.stem(token) for token in training]
    validation = [ps.stem(token) for token in validation]
    testing = [ps.stem(token) for token in testing]
    dictionary = [ps.stem(word) for word in dictionary]

dictionary.append(OOV)
dictionary.append(NUM)

# replacing OOV and NUM tokens
missing_tokens = set([token for token in training if token not in dictionary])
for (i, token) in enumerate(training):
    if token in missing_tokens:
        if token.isdigit():
            training[i] = NUM
        else:
            training[i] = OOV
            
missing_tokens = set([token for token in validation if token not in dictionary])
for (i, token) in enumerate(validation):
    if token in missing_tokens:
        if token.isdigit():
            validation[i] = NUM
        else:
            validation[i] = OOV

missing_tokens = set([token for token in testing if token not in dictionary])
for (i, token) in enumerate(testing):
    if token in missing_tokens:
        if token.isdigit():
            testing[i] = NUM
        else:
            testing[i] = OOV

### Training

In [56]:
def train_ngram_model(tokens):
    model = {}
    for token in tokens:
        model[token] = model.get(token, 0) + 1
    return model

In [57]:
from nltk import bigrams, trigrams

# unigram
unigram_model = train_ngram_model(training)

# bigram
bigram_model = train_ngram_model(list(bigrams(training)))

# unigram
trigram_model = train_ngram_model(list(trigrams(training)))

### Validation

In [ ]:
import math

from nltk import bigrams, trigrams

# unigram
result = 1 / len(validation) * sum([-math.log(unigram_model[token] / len(validation), 2) 
                                 for token in validation if token in unigram_model and unigram_model[token] > 0])
print 'Validation Unigram Cross-Entropy is:', '{0:.2f}'.format(result)

# bigram
bigram_tokens = list(bigrams(validation))
result = 1 / len(bigram_tokens) * sum([-math.log(bigram_model[token] / len(bigram_tokens), 2) 
                                 for token in bigram_tokens 
                                       if token in bigram_model and bigram_model[token] > 0])
print 'Validation Bigram Cross-Entropy is:', '{0:.2f}'.format(result)

trigram_tokens = list(trigrams(validation))
result = 1 / len(trigram_tokens) * sum([-math.log(trigram_model[token] / len(trigram_tokens), 2) 
                                 for token in trigram_tokens 
                                        if token in trigram_model and trigram_model[token] > 0])
print 'Validation Trigram Cross-Entropy is:', '{0:.2f}'.format(result)

### Testing

In [58]:
import math

from nltk import bigrams, trigrams

# unigram
result = 1 / len(testing) * sum([-math.log(unigram_model[token] / len(testing), 2) 
                                 for token in testing if token in unigram_model and unigram_model[token] > 0])
print 'Testing Unigram Cross-Entropy is:', '{0:.2f}'.format(result)

# bigram
bigram_tokens = list(bigrams(testing))
result = 1 / len(bigram_tokens) * sum([-math.log(bigram_model[token] / len(bigram_tokens), 2) 
                                 for token in bigram_tokens 
                                       if token in bigram_model and bigram_model[token] > 0])
print 'Testing Bigram Cross-Entropy is:', '{0:.2f}'.format(result)

trigram_tokens = list(trigrams(testing))
result = 1 / len(trigram_tokens) * sum([-math.log(trigram_model[token] / len(trigram_tokens), 2) 
                                 for token in trigram_tokens 
                                        if token in trigram_model and trigram_model[token] > 0])
print 'Testing Trigram Cross-Entropy is:', '{0:.2f}'.format(result)

Unigram Cross-Entropy is: 6.55
Bigram Cross-Entropy is: 8.01
Trigram Cross-Entropy is: 4.51
